In [14]:
import pandas as pd
import numpy as np
import urllib2 

In [3]:
# TODO: download from URL
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [4]:
with open(dropbox + "imdb/genres.list") as f:
    genres = f.readlines()
with open(dropbox + "imdb/plot.list") as f:
    plots = f.readlines()
with open(dropbox + "imdb/countries.list") as f:
    countries = f.readlines()

In [50]:
print len(genres)
print len(plots)
print len(countries)

2134860
6855909
1713923


In [8]:
plotlist = []
nextMovie = False
plot = []
revplots = []
nextPlot = True
by = False
for line in plots:
    #The titles that start with double quotes are TV shows, exclude them
    if line[:2] == 'MV' and line[4] == '"':
        nextMovie = False
        continue
    if line[:2] == 'MV':
        title = line[4:-1]
        i = title.find('(')
        j = title.find(')')
        if i > 0 and j > 0:
            year = title[i+1:j]
        nextMovie = True
    if nextMovie == True:
        revplots = []
        #instantiate a dict for the next movie
        movie = {'title':title, 'plots':revplots, 'year':year, 'type':'user plot', 'source':'imdb'}
        plotlist.append(movie)
        nextMovie = False
    if nextPlot == True:
        #instantiate a new plot
        plot = []
    if line[:2] == 'PL':
        #append to plot
        plot.append(line[4:].replace('\n',' '))
        nextPlot = False
        by = False
    if line[:2] == 'BY':
        #we're at the end of a plot, convert to string and append to list of plots for this review
        plotstr = ''.join(plot)
        revplots.append(plotstr.strip()) 
        nextPlot = True
        by = True
    if line[:2] == '--':
        if by == False:  #if there was no 'BY' line we append now
            plotstr = ''.join(plot)
            revplots.append(plotstr.strip())  
        nextPlot = True
        by = False
    

In [59]:
movieplots = pd.DataFrame(data=plotlist, columns=['title','source','year','plots', 'type'])

In [60]:
movieplots.head()

,title,source,year,plots,type
0,#1 Cheerleader Camp (2010) (V),imdb,2010,[When they're hired to work at a cheerleading ...,user plot
1,#1 Serial Killer (2013),imdb,2013,[Years of seething rage against the racism he'...,user plot
2,#1 at the Apocalypse Box Office (2015),imdb,2015,"[Jules is, self declared, the most useless per...",user plot
3,#137 (2011),imdb,2011,"[#137 is a SCI/FI thriller about a girl, Marla...",user plot
4,#29 (2012),imdb,2012,"[In #29, the constant zooming into certain lan...",user plot


In [62]:
movieplots.to_csv(dropbox + "movieplots.csv", index=False)

In [10]:
movieplots = pd.read_csv(dropbox + 'movieplots.csv')

476833
476836
476838
476841
476850
476852
476855
476856
476857
476859
476860
476864
476865
476866
476871
476872
476880
476882
476883
476886
476889
476897
476908
476910
476911
476913
476916
476922
476923
476926
476927
476928
476930
476934
476946
476953
476954
476957
476959
476966
476967
476971
476973
476975
476982
476988
476989
476990
476991
476992
476993
476995
476997
476998
477000
477003
477005
477006
477017
477018
477019
477072
477073
477080
477084
477088
477093
477102
477106
477107
477108
477109
477110
477111
477112
477113
477114
477117
477119
477121
477122
477124
477129
477130
477133
477142
477145
477149
477151
477153
477157
477164
477165
477168
477173
477177
477182
477184
477189
477192
477193
477199
477200
477204
477205
477206
477207
477211
477225
477226
477232
477235
477236
477239
477249
477252
477254
477256
477261
477262
477268
477269
477270
477273
477289
477295
477298
477305
477309
477319
477329
477337
477352
477353
477354
477355
477356
477357
477358
477359
477364
477371
477372